## Assignment 13 - 1

### Instructions
- OpenCV Yolo: [SOURCE](https://pysource.com/2019/06/27/yolo-object-detection-using-opencv-with-python/)
- Run this above code on your laptop or Colab. 
- Take an image of yourself, holding another object which is there in COCO data set (search for COCO classes to learn). 
- Run this image through the code above. 
- Upload the link to GitHub implementation of this
- Upload the annotated image by YOLO

### Reference
- [YOLO object detection using Opencv with Python](https://pysource.com/2019/06/27/yolo-object-detection-using-opencv-with-python/)



## Download files
- [yolov3.weights](https://pjreddie.com/media/files/yolov3.weights)
- [yolov3.cfg](https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg)
- [Class names - coco.names](https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names)

In [1]:
import cv2
import numpy as np

In [2]:
cv2.__version__

'4.1.2'

#### Display Sample Image

<img src=".\praveen-cup.png" style="zoom:40%;" />

### Define Constants

In [3]:
IMAGE_PATH = "praveen-cup.png"
YOLOV3_WEIGHTS = "yolov3.weights"
YOLOV3_CFG = "yolov3.cfg"
YOLOV3_CLASS_NAMES = "coco.names"

### Load Weights, Config and Classes

In [4]:
# load YOLO
net = cv2.dnn.readNet(YOLOV3_WEIGHTS, YOLOV3_CFG)
classes = []
with open(YOLOV3_CLASS_NAMES, "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

### Load image using cv2

In [5]:
# loading image
img = cv2.imread(IMAGE_PATH)
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

print(img.shape)

(266, 309, 3)


### Object Detection

In [6]:
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

### Display Results

In [7]:
# Showing information on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

#### Remove Noise using Non-max suppression

In [8]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

#### Display result on screen

In [ ]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 2)
        
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Save Predicted Image

In [ ]:
# save image
status = cv2.imwrite('praveen-cup-predicted.png', img)
 
print("Image written to file-system : ",status)

<img src=".\praveen-cup-predicted.png" style="zoom:100%;" />

## Analysis
- Image had two objects - Person and Cup
- Predicted is Person and Cake
- Wrong prediction for Cup, it has been predicted as Cake